In [14]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.layers import MaxPooling2D, Conv2D, GlobalAveragePooling2D
from file_processing import create_melspec, load_audio, load_files_list

In [15]:
melspec_params = {'n_mels': 128,'duration': 5*22050,'hop_length': 512,'n_fft': 2048,'fmin': 20}

In [16]:
path = '../Datasets/s_a_d__datasets/MiMii/normal_train/'
folders_list = load_files_list(path)
X = []

for folder in folders_list:
    sub_folder_list = load_files_list(path + folder)
    for sub_folder in sub_folder_list:
        file_list = load_files_list(path + folder + '/' + sub_folder)
        for file, i in zip(file_list, range(200)):
            audio, sr = load_audio(path + folder + '/' + sub_folder + '/' + file)
            melspec = create_melspec(melspec_params, audio, sr)
            X.append(melspec)


# X = np.asarray(X) # This is to solve a problem with the upscailings.
# Y = np.zeros([X.shape[0], X.shape[1], X.shape[2], X.shape[3] + 1, X.shape[4]])
# Y[:,:,:,:-1,:] = X
# X = Y
X.shape

AttributeError: 'list' object has no attribute 'shape'

In [17]:
X = np.asarray(X)
X.shape

(2253, 128, 216)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test = train_test_split(X, test_size=0.3, random_state=5)

In [20]:
X_train = X_train.reshape(X_train.shape[0], 128, 216, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 216, 1)

In [21]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, GlobalAveragePooling2D
from keras import Sequential
from keras.callbacks import EarlyStopping

In [24]:
autoencoder = Sequential()

autoencoder.add(Conv2D(8, (3,3), activation='relu', padding='same', input_shape=(128, 216, 1)))
autoencoder.add(MaxPool2D(pool_size=(2,2)))
autoencoder.add(Conv2D(16, (3,3), activation='relu', padding='same'))
autoencoder.add(MaxPool2D(pool_size=(2,2)))

autoencoder.add(Conv2D(32, (3,3), activation='relu', padding='same'))

autoencoder.add(UpSampling2D(size=2))
autoencoder.add(Conv2D(16, (3,3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D(size=2))
autoencoder.add(Conv2D(8,(3,3), activation='relu', padding='same'))

autoencoder.add(Conv2D(1,(3,3), activation='sigmoid', padding='same'))

In [25]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=5, verbose=1, mode='auto')]

In [26]:
history = autoencoder.fit(X_train, X_train, batch_size=45, epochs=50, callbacks=callbacks, validation_split=0.15)

Epoch 1/50
30/30 [==============================] - 34s 1s/step - loss: 1588.3552 - accuracy: 2.2430e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 2/50
30/30 [==============================] - 37s 1s/step - loss: 1587.9757 - accuracy: 2.4482e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 3/50
30/30 [==============================] - 36s 1s/step - loss: 1587.9756 - accuracy: 2.4482e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 4/50
30/30 [==============================] - 36s 1s/step - loss: 1587.9760 - accuracy: 2.4482e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 5/50
30/30 [==============================] - 36s 1s/step - loss: 1587.9760 - accuracy: 2.4482e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 6/50
30/30 [==============================] - 36s 1s/step - loss: 1587.9760 - accuracy: 2.4482e-05 - val_loss: 1623.4181 - val_accuracy: 2.2434e-05
Epoch 7/50
30/30 [==============================] - 36s 1s/step - loss: 1587